In [1]:
# Symlinkers
import sys
import os
import shutil
from pathlib import Path
from typing import Dict, Optional, Tuple
from attrs import define, field, Factory

@define(slots=False)
class SymlinkManager:
    """ stores references to multiple alternative versions of a filesystem directory (such as a .venv folder) and allows easily switching between these by modifying a symbolic link at a given location 
    
    
    """
    alternative_destination_directories: Dict[str, Path] = field() # the list of filesystem directories that will be symlinked to. The key is a shortname, otherwise the full path will be used.
    target_symlink_location: Path = field() # the location the symlink will be created
    

    def current_symlink_target(self) -> Tuple[Optional[str], Optional[Path]]:
        """ returns the current target the `target_symlink_location` points to. """
        if os.path.islink(self.target_symlink_location):
            found_target_path = Path(os.readlink(self.target_symlink_location)).resolve()
            found_known_destination_key = self.try_find_path_key_in_known_alternative_destination_directories(found_target_path)
            return (found_known_destination_key, found_target_path)
        else:
            return (None, None)

    def try_find_path_key_in_known_alternative_destination_directories(self, test_target_path: Path, debug_print=True) -> Optional[str]:
        found_known_destination_keys = [k for k, v in self.alternative_destination_directories.items() if v.resolve() == test_target_path]
        if len(found_known_destination_keys) > 0:
            if debug_print:
                print(f'equivalent to key: "{found_known_destination_keys[0]}"')
            return found_known_destination_keys[0]
        else:
            if debug_print:
                print(f'path "{test_target_path}" not found in self.alternative_destination_directories')
            return None # key not found
        
        
    def establish_symlink(self, new_target_path: Path):
        # Symlink the whl file to a generic version:
        new_target_path = new_target_path.resolve()
        found_known_destination_key = self.try_find_path_key_in_known_alternative_destination_directories(new_target_path)
        if found_known_destination_key is None:
            print(f'WARNING: new_target_path: "{new_target_path}" is not in self.alternative_destination_directories. Adding.')
            self.alternative_destination_directories[str(new_target_path)] = new_target_path


        symlink_path = self.target_symlink_location.resolve() # the path to the symlink
        # dst_path = 'current.whl'
        # Create the symbolic link
        try:
            print(f'\t symlinking {new_target_path} to {symlink_path}')
            os.symlink(new_target_path, symlink_path)
        except FileExistsError as e:
            print(f'\t WARNING: symlink {symlink_path} already exists. Removing it.')
            # Remove the symlink
            os.unlink(symlink_path)
            # Create the symlink
            os.symlink(new_target_path, symlink_path)
        except Exception as e:
            raise e
        

    @classmethod
    def version_real_folder(cls, real_folder_path: Path, destination_storage_parent_folder: Path, override_destination_storage_name: Optional[str]=None) -> "SymlinkManager":
        """ takes a real (non-symlink) folder path that will be moved to a different destination and then symlinked back to the directory with the same name. 
        
        """
        # Ensure that `real_folder_path` isn't already a symlink
        if os.path.islink(real_folder_path):
            raise ValueError("The provided path is already a symlink.")

        if override_destination_storage_name is None:
            override_destination_storage_name = real_folder_path.name
            
        destination_storage_folder: Path = destination_storage_parent_folder.joinpath(override_destination_storage_name).resolve()

        # destination_storage_folder should not already exist:
        assert (not destination_storage_folder.exists()), f"destination_storage_folder: {destination_storage_folder} already exists! Cannot symlink here. Specify a `override_destination_storage_name` if needed."
        
        # Make a full recursive copy of `real_folder_path` to the destination location (`destination_storage_folder`)
        shutil.copytree(real_folder_path, destination_storage_folder)

        # Replace the real directory at `real_folder_path` with a symlink back to the new_copy_destination
        if os.path.exists(real_folder_path):
            shutil.rmtree(real_folder_path)
        
        os.symlink(destination_storage_folder, real_folder_path)

        # Return a new SymlinkManager instance
        return cls(
            alternative_destination_directories={override_destination_storage_name: destination_storage_folder},
            target_symlink_location=real_folder_path
        )


_venv_symlinker = SymlinkManager(
    alternative_destination_directories={
        'pypoetry': Path(r'K:\FastSwap\Environments\pypoetry\pypoetry\Cache\virtualenvs\spike3d-UP7QTzFM-py3.9'),
        'original': Path(r'K:\FastSwap\Environments\.venv_original')
    },
    target_symlink_location=Path(r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv')
)
_venv_symlinker

_venv_symlinker.current_symlink_target()

equivalent to key: "pypoetry"


('pypoetry',
 WindowsPath('K:/FastSwap/Environments/pypoetry/pypoetry/Cache/virtualenvs/spike3d-UP7QTzFM-py3.9'))

In [ ]:
_venv_symlinker

In [3]:
## Copy
real_folder = Path(r"C:\Users\pho\.pyenv\pyenv-win\versions\3.9.13new").resolve()
destination_storage_parent_folder = Path(r'K:\FastSwap\Environments\pyenv\versions').resolve()

SymlinkManager.version_real_folder(real_folder, destination_storage_parent_folder=destination_storage_parent_folder, override_destination_storage_name='3.9.13new')

In [ ]:
'pyenv which python'
C:\Users\pho\.pyenv\pyenv-win\versions\3.9.13\python.exe


In [ ]:
# Windows:
'echo %PYENV_ROOT%\versions\'

# Linux:
echo "$PYENV_ROOT/versions/" 